#Import Indic NLP


In [ ]:
!unzip "Data.zip"

Archive:  Data.zip
  inflating: FinalHindiMRC.json      
  inflating: FinalMLQA.json          
  inflating: FinalXSQUAD.json        


In [ ]:
!git clone "https://github.com/anoopkunchukuttan/indic_nlp_library"
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
!pip install Morfessor
# The path to the local git repo for Indic NLP library
INDIC_NLP_LIB_HOME=r"/content/indic_nlp_library"

# The path to the local git repo for Indic NLP Resources
INDIC_NLP_RESOURCES="/content/indic_nlp_resources"

In [ ]:
import sys
sys.path.append(r'{}'.format(INDIC_NLP_LIB_HOME))
from indicnlp import common
common.set_resources_path(INDIC_NLP_RESOURCES)
from indicnlp import loader
loader.load()
from indicnlp.tokenize import indic_tokenize  

#**Prepare Data For Hindi MRC**

In [ ]:
import json
with open('FinalXSQUAD.json') as f:
  xquad = json.load(f)

In [ ]:
"""
xquad={}
k=0
for i in range(48):
    for x in xquad_dataset['data'][i]['paragraphs']:
        for y in x['qas']:
            xquad[str(k)]={}
            xquad[str(k)]['context']=x['context']
            xquad[str(k)]['question']=y['question']
            xquad[str(k)]['answer']=y['answers'][0]['text']
            k+=1

print(len(xquad))
"""

In [ ]:
i=0
for x in xquad:
    if(i<20):
        print(xquad[x])
        i+=1

In [ ]:
print(len(xquad))

# Indic Transformer.

In [ ]:
!pip install -q transformers
!pip install sentencepiece

In [ ]:
import torch
from transformers import pipeline
from transformers import AutoModelForQuestionAnswering, AutoTokenizer


#correct models

In [ ]:
model_name = "salti/bert-base-multilingual-cased-finetuned-squad"
qa_pipeline = pipeline(
    "question-answering",
    model=model_name,
    tokenizer=model_name
)

In [ ]:
context=xquad['10']['context']
question=xquad['10']['question']
print(xquad['10']['answer'])
result=qa_pipeline({

    
    'context': context,
    'question': question
    
})
print(result)

24
{'score': 0.7911100387573242, 'start': 72, 'end': 74, 'answer': '24'}


In [ ]:
salti_APA={}
model_name = "salti/bert-base-multilingual-cased-finetuned-squad"
qa_pipeline = pipeline(
    "question-answering",
    model=model_name,
    tokenizer=model_name
)

In [ ]:

i=1
for x in xquad:
    print(i)
    salti_APA[x]={}
    salti_APA[x]['actual']=xquad[x]['answer']#.replace(' ','')
    context=xquad[x]['context']
    question=xquad[x]['question']

    result=qa_pipeline({ 'context': context, 'question': question})
    answer=result['answer']#.replace(' ','')
    #answer=answer.replace(',','')
    salti_APA[x]['predicted']=answer
    #print(answer)
    i+=1



1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
print(salti_APA.keys())
for x in salti_APA:
    print(salti_APA[x]['actual'])


Calculate EM score

In [ ]:
 EM=0
 for x in salti_APA:
     if salti_APA[x]['actual']== salti_APA[x]['predicted']:
         EM+=1
print((EM/len(salti_APA))*100)

40.588235294117645


calculate F1 Score

In [ ]:
import collections
#comman=collections.Counter(actual_tok) & collections.Counter(predicted)

def f1(actual,predicted):
    actual_tok=[t for t in indic_tokenize.trivial_tokenize(actual)]
    pred_tok=[t for t in indic_tokenize.trivial_tokenize(predicted)]
    #print(actual,predicted)
    #print(actual_tok,pred_tok)
    #print(collections.Counter(actual_tok))
    #print(collections.Counter(pred_tok))
    #comman=collections.Counter(actual_tok) & collections.Counter(predicted)
    #print(comman)
    num_same=0
    for x in actual_tok:
        if x in pred_tok:
            num_same+=1
    #num_same=sum(x == y for x, y in zip(actual_tok, pred_tok))
    #print(num_same)
    if len(actual_tok)==0 or len(pred_tok)==0:
        return int(actual_tok == pred_tok)
    if num_same==0:
        return 0
    precision=1.0*num_same/len(pred_tok)
    recall=1.0*num_same/len(actual_tok)
    f1_score=2*precision*recall/(precision+recall)

    return f1_score

#print(salti_APA['0']['actual'],salti_APA['0']['predicted'])

In [ ]:
print(f1(salti_APA['1020']['actual'],salti_APA['1020']['predicted']))
for x in salti_APA:
    salti_APA[x]['f1_score']=f1(salti_APA[x]['actual'],salti_APA[x]['predicted'])

1.0


In [ ]:
avg_f1_score=0
for x in salti_APA:
    avg_f1_score+=salti_APA[x]['f1_score']

print((avg_f1_score/len(salti_APA))*100)

55.49557062064079


In [ ]:
with open("salti.json", "w") as outfile: 
    json.dump(salti_APA, outfile)

In [ ]:
import json
with open('salti.json') as f:
  salti_APA = json.load(f)

In [ ]:
EM=0
for x in salti_APA:
    if salti_APA[x]['actual']== salti_APA[x]['predicted']:
        EM+=1
print((EM/len(salti_APA))*100)

39.32773109243698


In [ ]:
qa_pipeline = pipeline(
    "question-answering",
    model="mrm8488/bert-multi-cased-finedtuned-xquad-tydiqa-goldp",
    tokenizer="mrm8488/bert-multi-cased-finedtuned-xquad-tydiqa-goldp"
)

In [ ]:
context=xquad['900']['context']
question=xquad['900']['question']
print(xquad['900']['answer'])
result=qa_pipeline({

    
    'context': context,
    'question': question
    
})
print(result)

डेविड बैंक्स
{'score': 0.45802947878837585, 'start': 354, 'end': 366, 'answer': 'डेविड बैंक्स'}


In [ ]:
model_name = "deepset/xlm-roberta-large-squad2"
qa_pipeline = pipeline(
    "question-answering",
    model=model_name,
    tokenizer=model_name
)

In [ ]:
context=xquad['199']['context']
question=xquad['199']['question']
print(xquad['199']['answer'])
result=qa_pipeline({

    
    'context': context,
    'question': question
    
})
print(result)

“हम याचक हैं”
{'score': 0.7633939385414124, 'start': 75, 'end': 89, 'answer': ' “हम याचक हैं”'}


In [ ]:
qa_pipeline = pipeline(
    "question-answering",
    model="mrm8488/bert-multi-cased-finetuned-xquadv1",
    tokenizer="mrm8488/bert-multi-cased-finetuned-xquadv1"
)

In [ ]:
context=xquad['101']['context']
question=xquad['101']['question']
print(xquad['101']['answer'])
result=qa_pipeline({

    
    #'context': "कोरोनावायरस पश्चिम में आतंक बो रहा है क्योंकि यह इतनी तेजी से फैलता है।",
    #'question': "कोरोनावायरस घबराहट कहां है?"
    'context' : context,
    'question' : question
    
})
print(result)

अफ्रानजी
{'score': 0.7512237429618835, 'start': 553, 'end': 561, 'answer': 'अफ्रानजी'}


#Incorrect models

In [ ]:
model_name = "mrm8488/distilbert-multi-finetuned-for-xqua-on-tydiqa"
qa_pipeline = pipeline(
    "question-answering",
    model=model_name,
    tokenizer=model_name
)

In [ ]:

context=xquad['77']['context']
question=xquad['77']['question']
print(xquad['77']['answer'])
result=qa_pipeline({

    
    'context': context,
    'question': question
    
})
print(result)

मोमस
{'score': 0.7847948670387268, 'start': 171, 'end': 206, 'answer': 'लियोन शिलर की संगीत थिएटर मेलोड्राम'}


In [ ]:
qa_pipeline = pipeline(
    "question-answering",
    model="mrm8488/xlm-multi-finetuned-xquadv1",
    tokenizer="mrm8488/xlm-multi-finetuned-xquadv1"
)

In [ ]:
context=xquad['100']['context']
question=xquad['100']['question']
print(xquad['100']['answer'])

result=qa_pipeline({

    
    'context': context,
    'question': question
    
})
print(result)

सेल्जुक तुर्क
{'score': 5.7342276704730466e-05, 'start': 476, 'end': 479, 'answer': 'थे।'}


In [ ]:


qa_pipeline = pipeline(
    "question-answering",
    model="neuralspace-reverie/indic-transformers-hi-distilbert",
    tokenizer="neuralspace-reverie/indic-transformers-hi-distilbert"
)



Some weights of the model checkpoint at neuralspace-reverie/indic-transformers-hi-distilbert were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at neuralspace-reverie/indic-transformers-hi-distilbert and are newly initialized: ['qa_outputs.weigh

In [ ]:
context=xquad['100']['context']
question=xquad['100']['question']
print(xquad['100']['answer'])
qa_pipeline({

    'context': context,
    'question': question
    
})


In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
tokenizer = AutoTokenizer.from_pretrained("neuralspace-reverie/indic-transformers-hi-distilbert")
model = AutoModelForQuestionAnswering.from_pretrained("neuralspace-reverie/indic-transformers-hi-distilbert")

text = r"""  वैज्ञानिक प्रयोग की सफलता ने मनुष्य की बुद्धि का अपूर्व विकास कर दिया है। द्वितीय महायुद्ध में एटम बम की शक्ति ने कुछ क्षणों में ही जापान की अजेय शक्ति को पराजित कर दिया। इस शक्ति की युद्धकालीन सफलता ने अमेरिका, रूस, ब्रिटेन, फ्रान्स आदि सभी देशों को ऐसे शस्त्रास्त्रों के निर्माण की प्रेरणा दी की सभी भयंकर और सर्वविनाशकारी शस्त्र बनाने लगे। अब सेना को पराजित करने तथा शत्रु-देश पर पैदल सेना द्वारा आक्रमण करने के लिए शस्त्र-निर्माण के स्थान पर देश के विनाश करने की दिशा में शस्त्रास्त्र बनने लगे है। इन हथियारों का प्रयोग होने पर शत्रु-देशों की अधिकांश जनता और संपत्ति थोड़े समय में ही नष्ट की जा सकेगी। चूँकि ऐसे शस्त्रास्त्र प्रायः सभी स्वतन्त्र देशों के संग्रहालयों में कुछ न कुछ आ गये है, अतः युद्ध की स्थिति में उनका प्रयोग भी अनिवार्य हो जायेगा।
            अतः दुनिया का सर्वनाश या अधिकांश नाश तो अवश्य ही हो जायेगा। इसलिए निः शस्त्रीकरण की योजनाएँ बन रही हैं। शस्त्रास्त्रों के निर्माण में जो दिशा अपनाई गई, उसी के अनुसार आज इतने उत्रत शस्त्रास्त्र बन गये हैं, जिनके प्रयोग से व्यापक विनाश आसन्न दिखाई पड़ता है। अब भी परीक्षणों की रोकथाम तथा बने शस्त्रों के प्रयोग के रोकने के मार्ग खोजे जा रहे हैं। इन प्रयासों के मूल में एक भयंकर आतंक और विश्व विनाश का भय कार्य कर रहा है।
            """

questions = [
    "इस गद्यांश का मूल कथ्य क्या है ?",
    "भयंकर विनाशकारी आधुनिक शस्त्रास्त्रों के बनाने की प्रेरणा किसने दी ?",
    "एटम बम की अपार शक्ति का प्रथम अनुभव कैसे हुआ ?",
]

for question in questions:
    inputs = tokenizer(question, text, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]

    outputs = model(**inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    answer_start = torch.argmax(
        answer_start_scores
    )  # Get the most likely beginning of answer with the argmax of the score
    answer_end = torch.argmax(answer_end_scores) + 1  # Get the most likely end of answer with the argmax of the score

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

    print(f"Question: {question}")
    print(f"Answer: {answer}")

In [ ]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained('neuralspace-reverie/indic-transformers-hi-bert')
model = AutoModel.from_pretrained('neuralspace-reverie/indic-transformers-hi-bert')

In [ ]:
print(model)

MODELS


#**Anup Run!**

In [1]:
!unzip "Data.zip"

Archive:  Data.zip
  inflating: FinalHindiMRC.json      
  inflating: FinalMLQA.json          
  inflating: FinalXSQUAD.json        


In [2]:
!pip install -q transformers
!pip install sentencepiece

     |████████████████████████████████| 2.1MB 7.0MB/s 
     |████████████████████████████████| 3.3MB 48.5MB/s 
     |████████████████████████████████| 901kB 35.4MB/s 
     |████████████████████████████████| 1.2MB 8.6MB/s 


In [3]:
!git clone "https://github.com/anoopkunchukuttan/indic_nlp_library"
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
!pip install Morfessor
# The path to the local git repo for Indic NLP library
INDIC_NLP_LIB_HOME=r"/content/indic_nlp_library"

# The path to the local git repo for Indic NLP Resources
INDIC_NLP_RESOURCES="/content/indic_nlp_resources"

Cloning into 'indic_nlp_library'...
remote: Enumerating objects: 1271, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 1271 (delta 50), reused 54 (delta 25), pack-reused 1178
Receiving objects: 100% (1271/1271), 9.56 MiB | 11.45 MiB/s, done.
Resolving deltas: 100% (654/654), done.
Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 133, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 133 (delta 0), reused 2 (delta 0), pack-reused 126
Receiving objects: 100% (133/133), 149.77 MiB | 30.26 MiB/s, done.
Resolving deltas: 100% (51/51), done.
Checking out files: 100% (28/28), done.


In [4]:
import torch
from transformers import pipeline
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
import sys
sys.path.append(r'{}'.format(INDIC_NLP_LIB_HOME))
from indicnlp import common
common.set_resources_path(INDIC_NLP_RESOURCES)
from indicnlp import loader
loader.load()
from indicnlp.tokenize import indic_tokenize  

In [16]:
import json
with open('FinalMLQA.json') as f:
  xquad = json.load(f)

In [17]:
print(len(xquad))

2250


In [31]:
salti_APA={}
model_name = "deepset/xlm-roberta-large-squad2"
qa_pipeline = pipeline(
    "question-answering",
    model=model_name,
    tokenizer=model_name
)

In [32]:
i=1
for x in xquad:
    print(i)
    salti_APA[x]={}
    salti_APA[x]['actual']=xquad[x]['answer']#.replace(' ','')
    context=xquad[x]['context']
    question=xquad[x]['question']

    result=qa_pipeline({ 'context': context, 'question': question})
    answer=result['answer']#.replace(' ','')
    #answer=answer.replace(',','')
    salti_APA[x]['predicted']=answer
    #print(answer)
    i+=1



1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [33]:
EM=0
for x in salti_APA:
     if salti_APA[x]['actual']== salti_APA[x]['predicted']:
         EM+=1
print((EM/len(salti_APA))*100)

1.3777777777777778


In [34]:
import collections
#comman=collections.Counter(actual_tok) & collections.Counter(predicted)

def f1(actual,predicted):
    actual_tok=[t for t in indic_tokenize.trivial_tokenize(actual)]
    pred_tok=[t for t in indic_tokenize.trivial_tokenize(predicted)]
    #print(actual,predicted)
    #print(actual_tok,pred_tok)
    #print(collections.Counter(actual_tok))
    #print(collections.Counter(pred_tok))
    #comman=collections.Counter(actual_tok) & collections.Counter(predicted)
    #print(comman)
    num_same=0
    for x in actual_tok:
        if x in pred_tok:
            num_same+=1
    #num_same=sum(x == y for x, y in zip(actual_tok, pred_tok))
    #print(num_same)
    if len(actual_tok)==0 or len(pred_tok)==0:
        return int(actual_tok == pred_tok)
    if num_same==0:
        return 0
    precision=1.0*num_same/len(pred_tok)
    recall=1.0*num_same/len(actual_tok)
    f1_score=2*precision*recall/(precision+recall)

    return f1_score

In [35]:
for x in salti_APA:
    salti_APA[x]['f1_score']=f1(salti_APA[x]['actual'],salti_APA[x]['predicted'])

In [36]:
avg_f1_score=0
for x in salti_APA:
    avg_f1_score+=salti_APA[x]['f1_score']

print((avg_f1_score/len(salti_APA))*100)

67.22041198836189


#**Ayush Run!**

In [37]:
!unzip "Data.zip"

Archive:  Data.zip
replace FinalHindiMRC.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [38]:
!pip install -q transformers
!pip install sentencepiece

In [39]:
!git clone "https://github.com/anoopkunchukuttan/indic_nlp_library"
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
!pip install Morfessor
# The path to the local git repo for Indic NLP library
INDIC_NLP_LIB_HOME=r"/content/indic_nlp_library"

# The path to the local git repo for Indic NLP Resources
INDIC_NLP_RESOURCES="/content/indic_nlp_resources"

fatal: destination path 'indic_nlp_library' already exists and is not an empty directory.
fatal: destination path 'indic_nlp_resources' already exists and is not an empty directory.


In [40]:
import torch
from transformers import pipeline
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
import sys
sys.path.append(r'{}'.format(INDIC_NLP_LIB_HOME))
from indicnlp import common
common.set_resources_path(INDIC_NLP_RESOURCES)
from indicnlp import loader
loader.load()
from indicnlp.tokenize import indic_tokenize  

In [41]:
import json
with open('FinalMLQA.json') as f:
  xquad = json.load(f)

In [42]:
salti_APA={}
model_name = "salti/bert-base-multilingual-cased-finetuned-squad"
qa_pipeline = pipeline(
    "question-answering",
    model=model_name,
    tokenizer=model_name
)

In [43]:
i=1
for x in xquad:
    print(i)
    salti_APA[x]={}
    salti_APA[x]['actual']=xquad[x]['answer']#.replace(' ','')
    context=xquad[x]['context']
    question=xquad[x]['question']

    result=qa_pipeline({ 'context': context, 'question': question})
    answer=result['answer']#.replace(' ','')
    #answer=answer.replace(',','')
    salti_APA[x]['predicted']=answer
    #print(answer)
    i+=1



1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [44]:
EM=0
 for x in salti_APA:
     if salti_APA[x]['actual']== salti_APA[x]['predicted']:
         EM+=1
print((EM/len(salti_APA))*100)

IndentationError: ignored

In [ ]:
import collections
#comman=collections.Counter(actual_tok) & collections.Counter(predicted)

def f1(actual,predicted):
    actual_tok=[t for t in indic_tokenize.trivial_tokenize(actual)]
    pred_tok=[t for t in indic_tokenize.trivial_tokenize(predicted)]
    #print(actual,predicted)
    #print(actual_tok,pred_tok)
    #print(collections.Counter(actual_tok))
    #print(collections.Counter(pred_tok))
    #comman=collections.Counter(actual_tok) & collections.Counter(predicted)
    #print(comman)
    num_same=0
    for x in actual_tok:
        if x in pred_tok:
            num_same+=1
    #num_same=sum(x == y for x, y in zip(actual_tok, pred_tok))
    #print(num_same)
    if len(actual_tok)==0 or len(pred_tok)==0:
        return int(actual_tok == pred_tok)
    if num_same==0:
        return 0
    precision=1.0*num_same/len(pred_tok)
    recall=1.0*num_same/len(actual_tok)
    f1_score=2*precision*recall/(precision+recall)

    return f1_score

In [ ]:
for x in salti_APA:
    salti_APA[x]['f1_score']=f1(salti_APA[x]['actual'],salti_APA[x]['predicted'])

In [ ]:
avg_f1_score=0
for x in salti_APA:
    avg_f1_score+=salti_APA[x]['f1_score']

print((avg_f1_score/len(salti_APA))*100)